<a href="https://colab.research.google.com/github/Clasikpaige/202/blob/master/language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

from datasets import load_dataset

# Download the WikiText dataset
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

# Print information about the dataset
print(wikitext)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [ ]:
!pip install transformers datasets

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Load the WikiText dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="wikitext-2-raw/wiki.train.raw",
    block_size=128
)
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="wikitext-2-raw/wiki.valid.raw",
    block_size=128
)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-training",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Define the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Save the model weights
model.save_pretrained("./gpt2-wikitext-model")

# Save tokenizer configuration
tokenizer.save_pretrained("./gpt2-wikitext-tokenizer")

# Optionally, save the model files to Hugging Face Hub
from transformers import HfFolder

# Create a folder to store the model files
hf_folder = HfFolder("./gpt2-wikitext-hf")
hf_folder.add("./gpt2-wikitext-model")
hf_folder.add("./gpt2-wikitext-tokenizer")
hf_folder.push_to_hub("gpt2-wikitext")

# Download the model files to your local machine
hf_folder.export_to_json("gpt2-wikitext-export.json")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ValueError: Input file path wikitext-2-raw/wiki.train.raw not found

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the WikiText dataset
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-training",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=wikitext["train"],
    eval_dataset=wikitext["validation"],
)

# Train the model
trainer.train()

# Save the model weights
model.save_pretrained("./gpt2-wikitext-model")

# Save tokenizer configuration
tokenizer.save_pretrained("./gpt2-wikitext-tokenizer")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=Tr

IndexError: Invalid key: 27086 is out of bounds for size 0

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the WikiText dataset
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

# Print dataset lengths for debugging
print(len(wikitext["train"]))
print(len(wikitext["validation"]))

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-training",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=wikitext["train"],
    eval_dataset=wikitext["validation"],
)

# Train the model
trainer.train()

# Save the model weights
model.save_pretrained("./gpt2-wikitext-model")

# Save tokenizer configuration
tokenizer.save_pretrained("./gpt2-wikitext-tokenizer")


36718
3760


IndexError: Invalid key: 27086 is out of bounds for size 0

In [ ]:
  pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
print(len(wikitext["train"]))
print(len(wikitext["validation"]))


36718
3760


In [ ]:
!pip install transformers datasets --upgrade

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the WikiText dataset
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-training",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=wikitext["train"],
    eval_dataset=wikitext["validation"],
)

# Train the model
trainer.train()

# Save the model weights
model.save_pretrained("./gpt2-wikitext-model")

# Save tokenizer configuration
tokenizer.save_pretrained("./gpt2-wikitext-tokenizer")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=Tr

IndexError: Invalid key: 27086 is out of bounds for size 0

In [ ]:
!pip install transformers datasets --upgrade

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the WikiText dataset
wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")

# Use a smaller subset for testing
train_subset = wikitext["train"][:100]
val_subset = wikitext["validation"][:10]

# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-wikitext-training",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=500,
    logging_first_step=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=val_subset,
)

# Train the model
trainer.train()

# Save the model weights
model.save_pretrained("./gpt2-wikitext-model")

# Save tokenizer configuration
tokenizer.save_pretrained("./gpt2-wikitext-tokenizer")


KeyError: 0

In [ ]:
KeyError                                  Traceback (most recent call last)

<ipython-input-2-6534ea236fe0> in <cell line: 44>()
     42
     43 # Train the model
---> 44 trainer.train()
     45
     46 # Save the model weights

6 frames

/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py in <listcomp>(.0)
     49                 data = self.dataset.__getitems__(possibly_batched_index)
     50             else:
---> 51                 data = [self.dataset[idx] for idx in possibly_batched_index]
     52         else:
     53             data = self.dataset[possibly_batched_index]

KeyError: 0

SyntaxError: invalid decimal literal (<ipython-input-3-c057d1043800>, line 3)